In [18]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

import os

# Credit Approval Data Set (Part 2) 
[Data Source](https://archive.ics.uci.edu/ml/datasets/Credit+Approval)

In [19]:
def download_credit_ds():
    if not os.path.exists("data"):
        os.makedirs("data")    

    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data"
    response = requests.get(url)
    if response.status_code != 200:
        raise ValueError

    with open("data/credit-approval.csv", 'wb') as file:
        for chunk in response:
            file.write(chunk)

In [20]:
#if you have not data locally, use this
#download_credit_ds()

credit = pd.read_csv("data/credit-approval.csv", header=None, na_values=['?'])
credit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [21]:
bool_mask_ft = { 't' : True, 'f' : False }
bool_mask_pm = { '+' : True, '-' : False }

credit[8] = credit[8].map(bool_mask_ft).astype('bool')
credit[9] = credit[9].map(bool_mask_ft).astype('bool')
credit[11] = credit[11].map(bool_mask_ft).astype('bool')
credit[15] = credit[15].map(bool_mask_pm).astype('bool')

In [22]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       678 non-null    object 
 1   1       678 non-null    float64
 2   2       690 non-null    float64
 3   3       684 non-null    object 
 4   4       684 non-null    object 
 5   5       681 non-null    object 
 6   6       681 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    bool   
 9   9       690 non-null    bool   
 10  10      690 non-null    int64  
 11  11      690 non-null    bool   
 12  12      690 non-null    object 
 13  13      677 non-null    float64
 14  14      690 non-null    int64  
 15  15      690 non-null    bool   
dtypes: bool(4), float64(4), int64(2), object(6)
memory usage: 67.5+ KB


In [23]:
print(credit[0].unique())
print(credit[3].unique())
print(credit[4].unique())
print(credit[5].unique())
print(credit[12].unique())

['b' 'a' nan]
['u' 'y' nan 'l']
['g' 'p' nan 'gg']
['w' 'q' 'm' 'r' 'cc' 'k' 'c' 'd' 'x' 'i' 'e' 'aa' 'ff' 'j' nan]
['g' 's' 'p']


In [24]:
credit[0] = credit[0].astype('category')
credit[3] = credit[3].astype('category')
credit[4] = credit[4].astype('category')
credit[5] = credit[5].astype('category')
credit[12] = credit[12].astype('category')